In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

In [2]:
# baseline model with dropout and data augmentation on the cifar10 dataset
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
 
# load train and test dataset
def load_dataset():
 # load dataset
 (trainX, trainY), (testX, testY) = cifar10.load_data()
 # one hot encode target values
 trainY = to_categorical(trainY)
 testY = to_categorical(testY)
 return trainX, trainY, testX, testY
 
# scale pixels
def prep_pixels(train, test):
 # convert from integers to floats
 train_norm = train.astype('float32')
 test_norm = test.astype('float32')
 # normalize to range 0-1
 train_norm = train_norm / 255.0
 test_norm = test_norm / 255.0
 # return normalized images
 return train_norm, test_norm
 
# define cnn model
def define_model():
    new_model = Sequential()
    
    model = tf.keras.applications.ResNet50(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=(32,32,3),
        pooling=None,
        classes=10,
       # classifier_activation="softmax",
    #    **kwargs
        )
    
    
    new_model.add(model)
    #3rd conv layer
    new_model.add(tf.keras.layers.Conv2D(32, 3, padding="same"))
    new_model.add(tf.keras.layers.BatchNormalization())
    new_model.add(tf.keras.layers.Activation("relu"))
    new_model.add(tf.keras.layers.MaxPooling2D(2, padding='same'))
    #4th conv layer
    new_model.add(tf.keras.layers.Conv2D(64, 3, padding="same"))
    new_model.add(tf.keras.layers.BatchNormalization())
    new_model.add(tf.keras.layers.Activation("relu"))
    new_model.add(tf.keras.layers.MaxPooling2D(2, padding='same'))
    #Flatten Layer
    new_model.add(tf.keras.layers.Flatten())

    new_model.add(tf.keras.layers.Dense(128, kernel_regularizer = tf.keras.regularizers.l2( l=0.01)))
    new_model.add(tf.keras.layers.BatchNormalization())
    new_model.add(tf.keras.layers.Activation("relu"))
    new_model.add(tf.keras.layers.Dropout(0.2))
    #new_model.add(tf.keras.layers.l2( l=0.01))
    #new_model.add(layers.GlobalAveragePooling2D())
    #output Dense Layer
    new_model.add(tf.keras.layers.Dense(10))
    new_model.add(tf.keras.layers.Activation('softmax'))
    adam = tf.keras.optimizers.Adam(lr=0.001)
    
    opt = SGD(lr=0.001, momentum=0.9)
    new_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return new_model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
 # plot loss
    model_trained = history
    accuracy = model_trained.history['accuracy']
    val_accuracy = model_trained.history['val_accuracy']
    loss = model_trained.history['loss']
    val_loss = model_trained.history['val_loss']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Acurácia de treino')
    plt.plot(epochs, val_accuracy, 'g', label='Acurácia de validação')
    plt.title('Acurácia de treino e validação')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, 'b', label='Perda no treino')
    plt.plot(epochs, val_loss, 'g', label='Perda na validação')
    plt.title('Perda na validação e treino')
    plt.legend()
    plt.show()
 
# run the test harness for evaluating a model
def run_test_harness(epochs, verb):
 # load dataset
 trainX, trainY, testX, testY = load_dataset()
 # prepare pixel data
 trainX, testX = prep_pixels(trainX, testX)
 # define model
 model = define_model()
 # create data generator
 datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
 # prepare iterator
 it_train = datagen.flow(trainX, trainY, batch_size=64)
 # fit model
 steps = int(trainX.shape[0] / 64)
 history = model.fit(it_train, steps_per_epoch=steps, epochs=epochs, validation_data=(testX, testY), verbose=verb)
 # evaluate model
 _, acc = model.evaluate(testX, testY, verbose=verb)
 print('> %.3f' % (acc * 100.0))
 # learning curves
 summarize_diagnostics(history)


In [3]:
epochs = 400
verb = 2

In [ ]:
trainX, trainY, testX, testY = load_dataset()
 # prepare pixel data
trainX, testX = prep_pixels(trainX, testX)
 # define model
model = define_model()
 # create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
 # prepare iterator
it_train = datagen.flow(trainX, trainY, batch_size=64)
 # fit model
steps = int(trainX.shape[0] / 64)
history = model.fit(it_train, steps_per_epoch=steps, epochs=epochs, validation_data=(testX, testY), verbose=verb)


Epoch 1/400


C:\Users\tmcle\.conda\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\tmcle\.conda\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


781/781 - 149s - loss: 2.1750 - accuracy: 0.4960 - val_loss: 3.3448 - val_accuracy: 0.2229 - 149s/epoch - 191ms/step
Epoch 2/400
781/781 - 104s - loss: 1.4836 - accuracy: 0.6776 - val_loss: 1.2851 - val_accuracy: 0.7209 - 104s/epoch - 134ms/step
Epoch 3/400
781/781 - 104s - loss: 1.1818 - accuracy: 0.7332 - val_loss: 1.0742 - val_accuracy: 0.7517 - 104s/epoch - 133ms/step
Epoch 4/400
781/781 - 104s - loss: 0.9836 - accuracy: 0.7658 - val_loss: 0.8756 - val_accuracy: 0.7873 - 104s/epoch - 133ms/step
Epoch 5/400
781/781 - 104s - loss: 0.8460 - accuracy: 0.7860 - val_loss: 0.8042 - val_accuracy: 0.7914 - 104s/epoch - 133ms/step
Epoch 6/400
781/781 - 104s - loss: 0.7345 - accuracy: 0.8050 - val_loss: 0.8262 - val_accuracy: 0.7697 - 104s/epoch - 133ms/step
Epoch 7/400
781/781 - 103s - loss: 0.6534 - accuracy: 0.8175 - val_loss: 0.6916 - val_accuracy: 0.7977 - 103s/epoch - 132ms/step
Epoch 8/400
781/781 - 103s - loss: 0.5909 - accuracy: 0.8284 - val_loss: 0.6402 - val_accuracy: 0.8098 - 103s

Epoch 65/400
781/781 - 103s - loss: 0.0815 - accuracy: 0.9749 - val_loss: 0.5861 - val_accuracy: 0.8548 - 103s/epoch - 132ms/step
Epoch 66/400
781/781 - 103s - loss: 0.0781 - accuracy: 0.9767 - val_loss: 0.6168 - val_accuracy: 0.8497 - 103s/epoch - 132ms/step
Epoch 67/400
781/781 - 103s - loss: 0.0757 - accuracy: 0.9773 - val_loss: 0.6130 - val_accuracy: 0.8543 - 103s/epoch - 132ms/step
Epoch 68/400
781/781 - 103s - loss: 0.0746 - accuracy: 0.9778 - val_loss: 0.6164 - val_accuracy: 0.8532 - 103s/epoch - 132ms/step
Epoch 69/400
781/781 - 103s - loss: 0.0706 - accuracy: 0.9790 - val_loss: 0.6140 - val_accuracy: 0.8546 - 103s/epoch - 132ms/step
Epoch 70/400
781/781 - 103s - loss: 0.0713 - accuracy: 0.9791 - val_loss: 0.6376 - val_accuracy: 0.8483 - 103s/epoch - 132ms/step
Epoch 71/400
781/781 - 103s - loss: 0.0720 - accuracy: 0.9786 - val_loss: 0.6132 - val_accuracy: 0.8566 - 103s/epoch - 132ms/step
Epoch 72/400
781/781 - 103s - loss: 0.0671 - accuracy: 0.9801 - val_loss: 0.6437 - val_acc

In [ ]:
 # evaluate model
_, acc = model.evaluate(testX, testY, verbose=verb)
print('Acurácia: %.3f' % (acc * 100.0))

In [ ]:
 # learning curves
summarize_diagnostics(history)

In [ ]:
model_name = f'cifar-resnet50-baseline-holdout-{epochs}epochs_all_transfer.npy'
model.save(model_name)

In [ ]:
# entry point, run the test harness
#run_test_harness(epochs=100, verb=1)